# LocalLLM LoRA Training - Google Colab

This notebook trains all 6 LoRA profiles for your LocalLLM system.

**Profiles**:
1. Career Advisor
2. Marketing Specialist
3. Website Builder
4. Android Developer
5. Backend Developer
6. Frontend Developer

**Estimated Time**: 2-4 hours on free T4 GPU

**Cost**: Free (Google Colab free tier)

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install required packages
!pip install -q transformers==4.36.0
!pip install -q peft==0.7.1
!pip install -q trl==0.7.10
!pip install -q bitsandbytes==0.41.3
!pip install -q accelerate==0.25.0
!pip install -q datasets==2.16.1

print("✅ All packages installed!")

## Step 2: Mount Google Drive (to save adapters)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
!mkdir -p /content/drive/MyDrive/localllm_adapters
print("✅ Google Drive mounted!")

## Step 3: Upload Datasets

**Option A**: Upload from your computer (click folder icon on left, drag & drop)

**Option B**: Download from GitHub (if you pushed them)

Upload these files to `/content/datasets/`:
- career_advisor_starter.jsonl
- marketing_specialist_starter.jsonl
- website_builder_starter.jsonl
- android_mobile_starter.jsonl
- backend_starter.jsonl
- frontend_starter.jsonl

In [ ]:
# Create datasets directory
!mkdir -p /content/datasets

# If you have a GitHub repo, uncomment and modify:
# !git clone https://github.com/Paulocadias/LocalLLM.git
# !cp LocalLLM/datasets/lora_profiles/*.jsonl /content/datasets/

# Check uploaded files
!ls -lh /content/datasets/
print("\n✅ Upload your 6 JSONL files to /content/datasets/ before continuing!")

## Step 4: Training Script

In [ ]:
%%writefile train_lora.py
import argparse
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

PROFILE_PROMPTS = {
    "career-advisor": "You are an expert career advisor specializing in tech careers, job transitions, and professional development.",
    "marketing-specialist": "You are an expert marketing strategist specializing in digital marketing, content strategy, and growth.",
    "website-builder": "You are an expert web designer and developer specializing in high-converting websites and landing pages.",
    "android": "You are an expert Android developer specializing in Kotlin and modern Android development practices.",
    "backend": "You are an expert backend developer specializing in API design, database optimization, and server-side architecture.",
    "frontend": "You are an expert frontend developer specializing in React, TypeScript, and modern web development."
}

def load_model_and_tokenizer(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    model = prepare_model_for_kbit_training(model)
    return model, tokenizer

def create_lora_config():
    return LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

def prepare_dataset(dataset_path, tokenizer, profile):
    dataset = load_dataset('json', data_files=dataset_path, split='train')
    system_prompt = PROFILE_PROMPTS.get(profile, "You are a helpful AI assistant.")
    
    def format_prompt(example):
        prompt = f"""<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{example['instruction']}"""
        if example.get('input'):
            prompt += f"\n\nContext: {example['input']}"
        prompt += f"""<|im_end|>\n<|im_start|>assistant\n{example['output']}<|im_end|>"""
        return {"text": prompt}
    
    return dataset.map(format_prompt, remove_columns=dataset.column_names)

def fine_tune_lora(profile, dataset_path, output_dir, model_name="Qwen/Qwen2.5-7B-Instruct", epochs=2, batch_size=1):
    logger.info(f"Starting LoRA fine-tuning for profile: {profile}")
    logger.info(f"Dataset: {dataset_path}")
    logger.info(f"Output: {output_dir}")
    
    # Load model
    model, tokenizer = load_model_and_tokenizer(model_name)
    
    # Configure LoRA
    lora_config = create_lora_config()
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # Prepare dataset
    dataset = prepare_dataset(dataset_path, tokenizer, profile)
    logger.info(f"Dataset size: {len(dataset)} examples")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        save_strategy="epoch",
        optim="paged_adamw_8bit",
        warmup_ratio=0.1,
    )
    
    # Train
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=2048,
        tokenizer=tokenizer,
    )
    
    logger.info("Starting training...")
    trainer.train()
    
    # Save adapter
    logger.info(f"Saving adapter to {output_dir}")
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    logger.info(f"✅ Training completed for {profile}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--profile", required=True)
    parser.add_argument("--dataset", required=True)
    parser.add_argument("--output", required=True)
    parser.add_argument("--model", default="Qwen/Qwen2.5-7B-Instruct")
    parser.add_argument("--epochs", type=int, default=2)
    parser.add_argument("--batch-size", type=int, default=1)
    args = parser.parse_args()
    
    fine_tune_lora(
        profile=args.profile,
        dataset_path=args.dataset,
        output_dir=args.output,
        model_name=args.model,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

## Step 5: Train All Profiles

**This will take 2-4 hours total on T4 GPU**

In [ ]:
# Profile configurations
profiles = [
    ("career-advisor", "career_advisor_starter.jsonl"),
    ("marketing-specialist", "marketing_specialist_starter.jsonl"),
    ("website-builder", "website_builder_starter.jsonl"),
    ("android", "android_mobile_starter.jsonl"),
    ("backend", "backend_starter.jsonl"),
    ("frontend", "frontend_starter.jsonl"),
]

# Train each profile
for profile_name, dataset_file in profiles:
    print(f"\n{'='*60}")
    print(f"Training: {profile_name}")
    print(f"{'='*60}\n")
    
    !python train_lora.py \
        --profile {profile_name} \
        --dataset /content/datasets/{dataset_file} \
        --output /content/drive/MyDrive/localllm_adapters/{profile_name} \
        --model Qwen/Qwen2.5-7B-Instruct \
        --epochs 2 \
        --batch-size 1
    
    print(f"\n✅ {profile_name} training complete!\n")

print("\n" + "="*60)
print("🎉 ALL PROFILES TRAINED SUCCESSFULLY!")
print("="*60)
print("\nAdapters saved to: /content/drive/MyDrive/localllm_adapters/")
print("\nDownload them to your local machine and place in:")
print("  C:\\BOT\\localLLM\\lora_adapters\\")

## Step 6: Download Adapters

Your trained adapters are saved in Google Drive at:
`/content/drive/MyDrive/localllm_adapters/`

Download each folder:
1. career-advisor
2. marketing-specialist
3. website-builder
4. android
5. backend
6. frontend

Place them in: `C:\BOT\localLLM\lora_adapters\` on your PC

## Step 7: Verify Adapters

In [ ]:
# Check all adapters were created
!ls -lh /content/drive/MyDrive/localllm_adapters/

# Check adapter contents
for profile in ["career-advisor", "marketing-specialist", "website-builder", "android", "backend", "frontend"]:
    print(f"\n{profile}:")
    !ls /content/drive/MyDrive/localllm_adapters/{profile}/ | head -5

## Next Steps (On Your Local PC)

1. **Download adapters** from Google Drive

2. **Place in**: `C:\BOT\localLLM\lora_adapters\`

3. **Create Modelfiles**:
```bash
cd scripts/lora_profiles
./create_modelfiles.sh
```

4. **Register with Ollama**:
```bash
./register_models.sh
```

5. **Test**:
```bash
ollama run qwen-career-advisor "How do I negotiate salary?"
```

Done! Your system now has 100% quality with LoRA specialization! 🚀